In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,2022-05-16 04:02:24,-23.1203,-134.9692,77.49,67,61,20.04,broken clouds
1,1,Hermanus,ZA,2022-05-16 04:02:46,-34.4187,19.2345,57.79,56,0,9.57,clear sky
2,2,Nanortalik,GL,2022-05-16 04:02:55,60.1432,-45.2371,30.88,83,89,7.36,overcast clouds
3,3,Abu Dhabi,AE,2022-05-16 04:05:30,24.4667,54.3667,84.31,66,0,5.75,clear sky
4,4,Avarua,CK,2022-05-16 04:03:15,-21.2078,-159.7750,76.96,68,100,8.52,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 75
What is your desired maximum temperature for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,2022-05-16 04:02:24,-23.1203,-134.9692,77.49,67,61,20.04,broken clouds
3,3,Abu Dhabi,AE,2022-05-16 04:05:30,24.4667,54.3667,84.31,66,0,5.75,clear sky
4,4,Avarua,CK,2022-05-16 04:03:15,-21.2078,-159.7750,76.96,68,100,8.52,overcast clouds
8,8,Saint George,US,2022-05-16 04:02:49,37.1041,-113.5841,82.99,14,0,5.75,clear sky
10,10,Kavieng,PG,2022-05-16 04:02:34,-2.5744,150.7967,83.08,69,94,1.97,light rain
...,...,...,...,...,...,...,...,...,...,...,...
666,666,Arlit,NE,2022-05-16 04:06:34,18.7369,7.3853,78.46,17,1,6.44,clear sky
669,669,Princeton,US,2022-05-16 04:13:46,25.5384,-80.4089,81.01,92,0,5.75,clear sky
670,670,Hami,CN,2022-05-16 04:05:43,42.8000,93.4500,79.72,4,100,6.55,overcast clouds
671,671,Juegang,CN,2022-05-16 04:13:47,32.3174,121.1855,78.94,36,2,5.32,clear sky


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,2022-05-16 04:02:24,-23.1203,-134.9692,77.49,67,61,20.04,broken clouds
3,3,Abu Dhabi,AE,2022-05-16 04:05:30,24.4667,54.3667,84.31,66,0,5.75,clear sky
4,4,Avarua,CK,2022-05-16 04:03:15,-21.2078,-159.7750,76.96,68,100,8.52,overcast clouds
8,8,Saint George,US,2022-05-16 04:02:49,37.1041,-113.5841,82.99,14,0,5.75,clear sky
10,10,Kavieng,PG,2022-05-16 04:02:34,-2.5744,150.7967,83.08,69,94,1.97,light rain
...,...,...,...,...,...,...,...,...,...,...,...
666,666,Arlit,NE,2022-05-16 04:06:34,18.7369,7.3853,78.46,17,1,6.44,clear sky
669,669,Princeton,US,2022-05-16 04:13:46,25.5384,-80.4089,81.01,92,0,5.75,clear sky
670,670,Hami,CN,2022-05-16 04:05:43,42.8000,93.4500,79.72,4,100,6.55,overcast clouds
671,671,Juegang,CN,2022-05-16 04:13:47,32.3174,121.1855,78.94,36,2,5.32,clear sky


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = " "
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,77.49,broken clouds,-23.1203,-134.9692,
3,Abu Dhabi,AE,84.31,clear sky,24.4667,54.3667,
4,Avarua,CK,76.96,overcast clouds,-21.2078,-159.7750,
8,Saint George,US,82.99,clear sky,37.1041,-113.5841,
10,Kavieng,PG,83.08,light rain,-2.5744,150.7967,
11,Dingle,PH,88.83,overcast clouds,10.9995,122.6711,
32,Hithadhoo,MV,83.01,overcast clouds,-0.6000,73.0833,
33,Oistins,BB,80.69,broken clouds,13.0667,-59.5333,
34,Tautira,PF,81.27,light rain,-17.7333,-149.1500,
36,Mogadishu,SO,81.01,clear sky,2.0371,45.3438,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")  

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,77.49,broken clouds,-23.1203,-134.9692,People ThankYou
3,Abu Dhabi,AE,84.31,clear sky,24.4667,54.3667,Beach Rotana
4,Avarua,CK,76.96,overcast clouds,-21.2078,-159.7750,Paradise Inn
8,Saint George,US,82.99,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
10,Kavieng,PG,83.08,light rain,-2.5744,150.7967,Nusa Island Retreat
...,...,...,...,...,...,...,...
666,Arlit,NE,78.46,clear sky,18.7369,7.3853,Case de Passage De la Prefecture
669,Princeton,US,81.01,clear sky,25.5384,-80.4089,American Best Inn
670,Hami,CN,79.72,overcast clouds,42.8000,93.4500,Laoyutang Resort
671,Juegang,CN,78.94,clear sky,32.3174,121.1855,Rudong Y Hotel


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig 

Figure(layout=FigureLayout(height='420px'))

SyntaxError: invalid syntax (Temp/ipykernel_52820/2753235763.py, line 2)

<Figure size 432x288 with 0 Axes>